In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from celluloid import Camera
from IPython.display import HTML

In [3]:

class TuringMachine:
    def __init__(self, tape, blank_symbol, initial_state, final_states, transition_function):
        self.tape = list(tape)
        self.blank_symbol = blank_symbol
        self.head_position = 0
        self.current_state = initial_state
        self.final_states = final_states
        self.transition_function = transition_function

    def _print_tape(self, with_index: bool = False):
        tape_len = len(self.tape)
        print(f'\ttape:  [{", ".join([f'{cell}'.ljust(len(str(tape_len))) for cell in self.tape])}]')
        if with_index:
            print(f'\tindex: [{", ".join([f'{i}'.ljust(len(str(tape_len))) for i in range(tape_len)])}]')

    def step(self, debug: bool = False):
        current_symbol = self.tape[self.head_position]
        if (self.current_state, current_symbol) in self.transition_function:
            new_symbol, direction, new_state = self.transition_function[(self.current_state, current_symbol)]
            if debug:
                print(f"{self.head_position} -> {self.head_position + (1 if direction == 'R' else -1)} | state: {self.current_state} -> {new_state}, symbol: {current_symbol} -> {new_symbol} | dir: {direction}")
                # print(f"\ttape: {self.tape}")
                self._print_tape()
            self.tape[self.head_position] = new_symbol
            if debug:
                # print(f"\ttape: {self.tape}")
                self._print_tape(with_index=True)
            self.head_position += 1 if direction == 'R' else -1
            self.current_state = new_state
        else:
            raise Exception(f"No transition defined for this state ({self.current_state}) and symbol ({current_symbol})")

    def run(self, debug: bool = False):
        self.step(debug) # do ... while (so we can interrupt at the right time)
        while self.current_state not in self.final_states:
            self.step(debug)


In [4]:
# Turing machine code-generator functions

symbols = [
    0, 1,
    '00', '01', '10', '11', # intermediate states (current/next)
    # marked cells
    'O', 'X',
    'OO', 'OX', 'XO', 'XX', # marked intermediate states (current/next)
    # marked center cells
    'CO', 'CX',
    'COO', 'COX', 'CXO', 'CXX', # marked center intermediate states (current/next)
    # tape boundaries
    '^', '$',
    # grid boundaries
    'E', 'R'
]

def _tm_reljump(name: str, size: int, end_state: str, stop_symbols: list[int | str] = []):
    """
    relative jump for a turing machine
    in normal computer terms:
    pc <- pc + size
    state <- end_state
    """

    dir = 'R' if size > 0 else 'L'
    rev_dir = 'L' if dir == 'R' else 'R'
    # Don't jump beyond the tape
    disallowed_symbols = ['^' if dir == 'L' else '$'] + stop_symbols
    allowed_symbols = [s for s in symbols if s not in disallowed_symbols]

    # zero-size jump is a bit tricky:
    if size == 0:
        return {
            **{(f'{name}_0', s): (s, 'R', f'{name}_1') for s in allowed_symbols},
            **{(f'{name}_1', s): (s, 'L', end_state) for s in allowed_symbols},
        }
    
    # TODO: handle OOB state (end at a different state at the end of the tape.)
    states = {}

    # Pre-end state (only when we hit a disallowed symbol)
    for symbol in allowed_symbols:
        states[(f'{name}_preend', symbol)] = (symbol, dir, end_state)

    for i in range(abs(size) - 1):
        for s in allowed_symbols:
            states[(f'{name}_{i}', s)] = (s, dir, f'{name}_{i+1}')
        for s in disallowed_symbols:
            states[(f'{name}_{i}', s)] = (s, rev_dir, f'{name}_preend')
    for s in allowed_symbols:
        states[(f'{name}_{abs(size) - 1}', s)] = (s, dir, end_state)
    for s in disallowed_symbols:
        states[(f'{name}_{abs(size) - 1}', s)] = (s, rev_dir, f'{name}_preend')
    # states[(f'{name}_{abs(size) - 1}', 0)] = (0, dir, end_state)
    # states[(f'{name}_{abs(size) - 1}', 1)] = (1, dir, end_state)
    return states

def _tm_fwd_symjmp(name: str, symbol: int | str | list[int | str], end_state: str):
    """
    jumps forward until a certain symbol is found
    """
    stop_symbols = symbol if isinstance(symbol, list) else [symbol]
    skip_symbols = [s for s in symbols if s not in stop_symbols] 

    states = {
        **{(f'{name}', s): (s, 'R', f'{name}') for s in skip_symbols},
        **{(f'{name}', s): (s, 'L', f'{name}_end') for s in stop_symbols},
        **{(f'{name}_end', s): (s, 'R', end_state) for s in symbols},
    }
    return states

def _tm_bwd_symjmp(name: str, symbol: int | str | list[str | int], end_state: str):
    """
    jumps backward until a certain symbol is found
    """
    stop_symbols = symbol if isinstance(symbol, list) else [symbol]
    skip_symbols = [s for s in symbols if s not in stop_symbols] 
    states = {
        **{(f'{name}', s): (s, 'L', f'{name}') for s in skip_symbols},
        **{(f'{name}', s): (s, 'R', f'{name}_end') for s in stop_symbols},
        **{(f'{name}_end', s): (s, 'L', end_state) for s in symbols},
    }
    return states

def _tm_increment(name: str, bit_size: int):
    """
    increments a binary number with bit_size bits
    starts at the number's least significant bit, and assumes MSB storage (MSB is the leftmost bit)
    does not handle overflow - i.e. the number rolls over to 0
    starting-state:
        {name}_0
    ending-state:
        {name}_end (same position as the starting state)
    """
    states = {
        # end-state
        (f"{name}_end", 0): (0, 'L', f"{name}_end"),
    }


    for i in range(bit_size):
        # generate return states
        if i > 0:
            for s in symbols:
                states[(f"{name}_{i}_ret", s)] = (s, 'R', f"{name}_{i - 1}_ret")
        else:
            for s in symbols:
                states[(f"{name}_{i}_ret", s)] = (s, 'R', f"{name}_end")

        
        # if the current bit is 0, set it to 1 and jump to the end of the number
        states[(f"{name}_{i}", 0)] = (1, 'L', f"{name}_{i}_ret")

        # if the current bit is 1, set it to 0 and move to the next bit
        # unless it's the last bit, in which case we're done
        if i == bit_size - 1:
            states[(f"{name}_{i}", 1)] = (0, 'L', f"{name}_{i}_ret")
        else:
            states[(f"{name}_{i}", 1)] = (0, 'L', f"{name}_{i + 1}")
    return states

def _tm_decrement(name: str, bit_size: int):
    """
    decrements a binary number with bit_size bits
    starts at the number's least significant bit, and assumes MSB storage (MSB is the leftmost bit)
    does not handle overflow - i.e. the number rolls over to 2**bit_size - 1
    starting-state:
        {name}_0
    ending-state:
        {name}_end (same position as the starting state)
    """
    states = {
        # pre-end state (because )
        (f"{name}_preend", 0): (0, 'R', f"{name}_end"),
        (f"{name}_preend", 1): (1, 'R', f"{name}_end"),
    }

    # algorithm works like this:
    # advance until the first 1 is found (or to the end of the number)
    # once it's found, set it to 0 and set all bits to the left of it to 1 (in the return-state(s))
    for i in range(bit_size):
        # if the current bit is set to 1, we can start going backwards
        if i > 0:
            states[(f'{name}_{i}', 1)] = (0, 'R', f'{name}_{i - 1}_ret')
        else:
            states[(f'{name}_{i}', 1)] = (0, 'L', f'{name}_preend')

        # if the current bit is 0
        # if it's the last bit, we're done
        # otherwise we advance to the next bit
        if i == bit_size - 1:
            states[(f'{name}_{i}', 0)] = (1, 'R', f"{name}_{i - 1}_ret")
        else:
            states[(f'{name}_{i}', 0)] = (0, 'L', f'{name}_{i + 1}')
        
        # generate return states
        if i > 0:
            states[(f'{name}_{i}_ret', 0)] = (1, 'R', f'{name}_{i - 1}_ret')
            # The case for 1 does not exist, since it should never happen (i.e. we let the tm error out)
        else:
            states[(f'{name}_{i}_ret', 0)] = (1, 'L', f"{name}_preend")
            # The case for 1 does not exist, since it should never happen (i.e. we let the tm error out)
    return states

def _tm_center_mark(state: str, next_state: str):
    states = {
        (state, 0): ('CO', 'R', next_state),
        (state, 1): ('CX', 'R', next_state),
        (state, '00'): ('COO', 'R', next_state),
        (state, '01'): ('COX', 'R', next_state),
        (state, '10'): ('CXO', 'R', next_state),
        (state, '11'): ('CXX', 'R', next_state),
    }
    return states

def _tm_mark(state: str, next_state: str):
    states = {
        (state, 0): ('O', 'R', next_state),
        (state, 1): ('X', 'R', next_state),
        (state, '00'): ('OO', 'R', next_state),
        (state, '01'): ('OX', 'R', next_state),
        (state, '10'): ('XO', 'R', next_state),
        (state, '11'): ('XX', 'R', next_state),
    }
    return states

def _tm_any_transition(from_state: str, to_state: str, dir: str = 'R'):
    states = {
        (from_state, s): (s, dir, to_state) for s in symbols
    }
    return states

In [ ]:
# Tests for the tm-codegen functions

# Implicitly tests the relative jump function as well
def test_tm_decrement(size: int):
    for initial in tqdm(range(2 ** size), desc="Testing decrement"):
        name = 'dec'

        tape = [int(e) for e in bin(initial)[2:].zfill(size)]
        blank_symbol = ' '
        initial_state = 'start_0'
        final_states = {f'{name}_end'}
        
        transition_function = {
            **_tm_reljump('start', size - 1, f'{name}_0'),
            **_tm_decrement(name, size)
        }
        tm = TuringMachine(tape, blank_symbol, initial_state, final_states, transition_function)
        tm.run(debug=False)
        result = int(''.join(map(str, tm.tape)), 2)
        # print(f"Initial: {initial}, Result: {result}")
        expected = initial - 1 if initial > 0 else 2 ** size - 1
        assert result == expected, f"Given: {initial}, Expected: {expected}, Got: {result}"
        assert tm.current_state == 'dec_end', f"Expected final state to be '{name}_end', got {tm.current_state}"
        assert tm.head_position == size - 1, f"Expected head position to be at the end of the tape, got {tm.head_position}"
    
    print(f"Turing machine decrement test passed, for size {size}")

# Implicitly tests the relative jump function as well
def test_tm_increment(size: int):
    for initial in tqdm(range(2 ** size), desc="Testing increment"):
        name = 'inc'

        tape = [int(e) for e in bin(initial)[2:].zfill(size)]
        blank_symbol = ' '
        initial_state = 'start_0'
        final_states = {f'{name}_end'}
        
        transition_function = {
            **_tm_reljump('start', size - 1, f'{name}_0'),
            **_tm_increment(name, size)
        }
        tm = TuringMachine(tape, blank_symbol, initial_state, final_states, transition_function)
        tm.run(debug=False)
        result = int(''.join(map(str, tm.tape)), 2)
        # print(f"Initial: {initial}, Result: {result}")
        expected = initial + 1 if initial < 2 ** size - 1 else 0
        assert result == expected, f"Given: {initial}, Expected: {expected}, Got: {result}"
        assert tm.current_state == 'inc_end', f"Expected final state to be '{name}_end', got {tm.current_state}"
        assert tm.head_position == size - 1, f"Expected head position to be at the end of the tape, got {tm.head_position}"
    
    print(f"Turing machine increment test passed, for size {size}")

test_tm_decrement(8)
test_tm_increment(8)

In [6]:
def get_initial_state(size: int):
    # last element is a state-bit (0 = running, 1 = halt)
    state = [0 for _ in range(size * size + 1)]

    # draw the glider
    state[0] = 0
    state[1] = 0
    state[2] = 1

    state[size + 0] = 1
    state[size + 1] = 0
    state[size + 2] = 1

    state[2 * size + 0] = 0
    state[2 * size + 1] = 1
    state[2 * size + 2] = 1

    return state

def visualize_state(state: list[int], size: int):
    state = np.array(state[:size * size]).reshape(size, size)
    plt.imshow(state, cmap='gray')
    plt.show()

def game_of_life_step(state: list[int], size: int):
    """
    A simple python-based implementation of the game of life rules.
    used as a check for correctness of the turing machine implementation.
    """
    new_state = [0 for _ in range(len(state))]
    for i in range(size):
        for j in range(size):
            # count the neighbors
            count = 0
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    if i + dx < 0 or i + dx >= size or j + dy < 0 or j + dy >= size:
                        continue
                    count += state[(i + dx) * size + j + dy]

            # update the cell
            if state[i * size + j] == 1:
                if count < 2 or count > 3:
                    new_state[i * size + j] = 0
                else:
                    new_state[i * size + j] = 1
            else:
                if count == 3:
                    new_state[i * size + j] = 1
                else:
                    new_state[i * size + j] = 0

    return new_state

In [18]:
def create_tm_gol_tape(size: int, grid: list[int]) -> list[str | int]:
    tape = ['^']
    for i in range(size):
        row = grid[i * size:(i + 1) * size]
        tape += row
        if i < size - 1:
            tape.append('R')
    tape += ['E', 0, 0, 0, '$']
    return tape

def visualize_tm_gol_state(tape: list[str | int], size: int, show: bool = False) -> list[int]:
    grid = []
    for i in range(size):
        row = tape[1 + i * (size + 1):1 + (i + 1) * (size + 1)][:-1]
        grid += row

    grid = np.array(grid).reshape(size, size)
    plt.imshow(1.0 - grid, cmap='gray')
    if show:
        plt.show()

def visualize_tm_gol_state_intermediate(tape: list[str | int], size: int, show: bool = False) -> list[int]:
    grid = []
    for i in range(size):
        row = tape[1 + i * (size + 1):1 + (i + 1) * (size + 1)][:-1]
        # replace stuff in the row
        row = [[1.0, 1.0, 1.0] if cell in [0, '00', '01'] else cell for cell in row] # replace off states with white
        row = [[0.0, 0.0, 0.0] if cell in [1, '10', '11'] else cell for cell in row] # replace on states with black
        row = [[0.0, 0.0, 1.0] if cell in ['CO', 'CX'] else cell for cell in row] # replace center cells with blue
        row = [[1.0, 0.0, 0.0] if cell in ['X', 'XX', 'XO'] else cell for cell in row] # replace marked, and on cells with red
        row = [[0.0, 1.0, 0.0] if cell in ['O', 'OO', 'OX'] else cell for cell in row] # replace marked, and off cells with green
        grid += row
    grid = np.array(grid).reshape(size, size, 3)
    plt.imshow(grid)
    if show:
        plt.show()

In [41]:
# X replaces a 1, O replaces a 0, CX, CO replace the center cell

# cell neighborhood:
# 0 1 2 n
# 3 X 4 n
# 5 6 7 n

# we need better edge-detection
# if the symbol before C is ^ or R, we're at the left edge of the grid
# if the symbol after C is E or R, we're at the right edge of the grid

# the other two cases (top and bottom) are a bit more tricky
# basically, there's no good way, except a fwd and bwd-jump to one the start/end of the row

# TODO: do we actually need to mark the zero neighbors?
# TODO: make transition shorthand


def _make_tm_gol(size: int) -> dict:

    states = {
        # entry-point for testing
        # **_tm_reljump('start', size + 2, 'nb_mark_start'), # jump to the first cell of the first row
        **_tm_bwd_symjmp('loop_start', '^', 'start_0'), # jump to the start of the tape
        ('start_0', '^'): ('^', 'R', 'nb_mark_start'), # start the loop on the first cell of the first row

        # fn nb_mark_start
            # Function to mark the neighbors before counting them
            ('nb_mark_start', 'E'): ('E', 'L', 'cell_collapse'), # end of the grid, collapse the intermediates
            ('nb_mark_start', 'R'): ('R', 'R', 'nb_mark_start'), # skip over the row-end marker

            ('nb_mark_start', 0): (0, 'L', 'nb_mark_3'), # go to neighbor 3
            ('nb_mark_start', 1): (1, 'L', 'nb_mark_3'),

            # check if we're at the top-left edge of the grid
            ('nb_mark_3', '^'): ('^', 'R', 'nb_mark_3_periodic_0'), # this means we're at the top-left edge of the world
            # periodic boundary conditions
                **_tm_reljump('nb_mark_3_periodic', size - 1, 'nb_mark_3_wrap'),
                **_tm_mark('nb_mark_3_wrap', 'nb_mark_3_wrap_ret_0'),
                **_tm_reljump('nb_mark_3_wrap_ret', -size, 'nb_mark_c_noprev'),
            # if (top-left edge):
                # center cell (no previous row case)
                **_tm_center_mark('nb_mark_c_noprev', 'nb_mark_4_noprev'),
                
                # neighbor 4 (no previous row case)
                ('nb_mark_4_noprev', 'R'): ('R', 'R', 'nb_mark_4_noprev_periodic_0'),
                # periodic boundary conditions
                    **_tm_reljump('nb_mark_4_noprev_periodic', -size - 1, 'nb_mark_4_noprev_periodic_wrap'),
                    **_tm_mark('nb_mark_4_noprev_periodic_wrap', 'nb_mark_4_noprev_periodic_wrap_ret_0'),
                    **_tm_reljump('nb_mark_4_noprev_periodic_wrap_ret', size, 'nb_mark_jmp_prev_row_noprev_0'),
                **_tm_mark('nb_mark_4_noprev', 'nb_mark_jmp_prev_row_noprev_0'),

                # jump to the last row (it becomes the first under periodic boundary conditions)
                **_tm_reljump('nb_mark_jmp_prev_row_noprev', (size*size) - 1 - 3, 'nb_mark_0_noprev'),
                
                # neighbor 0 (no previous row case)
                ('nb_mark_0_noprev', 'R'): ('R', 'R', 'nb_mark_0_noprev_periodic_0'),
                # periodic boundary conditions
                    **_tm_reljump('nb_mark_0_noprev_periodic', size - 1, 'nb_mark_0_noprev_periodic_wrap'),
                    **_tm_mark('nb_mark_0_noprev_periodic_wrap', 'nb_mark_0_noprev_periodic_wrap_ret_0'),
                    **_tm_reljump('nb_mark_0_noprev_periodic_wrap_ret', -size, 'nb_mark_1_noprev'),
                **_tm_mark('nb_mark_0_noprev', 'nb_mark_1_noprev'),

                # neighbor 1 (no previous row case)
                **_tm_mark('nb_mark_1_noprev', 'nb_mark_2_noprev'),

                # neighbor 2 (no previous row case)
                **_tm_mark('nb_mark_2_noprev', 'nb_mark_2_noprev_return_0'),
                # periodic boundary conditions
                ('nb_mark_2_noprev', 'E'): ('E', 'R', 'nb_mark_2_noprev_periodic_0'),
                    **_tm_reljump('nb_mark_2_noprev_periodic', -size - 1, 'nb_mark_2_noprev_periodic_mark'),
                    **_tm_mark('nb_mark_2_noprev_periodic_mark', 'nb_mark_2_noprev_periodic_ret_0'),
                    **_tm_reljump('nb_mark_2_noprev_periodic_ret', size, 'nb_mark_2_noprev_return_0'),
                **_tm_reljump('nb_mark_2_noprev_return', -(size*size) + 1 - 3, 'nb_mark_jmp_next_row_noprev_0'),


                # return back to the next row, i.e. the row after the first
                **_tm_reljump('nb_mark_jmp_next_row_noprev', size + 1, 'nb_mark_5_noprev'),

                # neighbor 5 (no previous row case)
                ('nb_mark_5_noprev', 'R'): ('R', 'R', 'nb_mark_5_noprev_periodic_0'),
                # periodic boundary conditions
                    **_tm_reljump('nb_mark_5_noprev_periodic', size - 1, 'nb_mark_5_noprev_wrap'),
                    **_tm_mark('nb_mark_5_noprev_wrap', 'nb_mark_5_noprev_wrap_ret_0'),
                    **_tm_reljump('nb_mark_5_noprev_wrap_ret', -size, 'nb_mark_6_noprev'),
                **_tm_mark('nb_mark_5_noprev', 'nb_mark_6_noprev'),

                # neighbor 6 (no previous row case)
                **_tm_mark('nb_mark_6_noprev', 'nb_mark_7_noprev'),

                # neighbor 7 (no previous row case) + jump to neighbor counting fn
                ('nb_mark_7_noprev', 'R'): ('R', 'R', 'nb_mark_7_noprev_periodic_0'),
                    # periodic boundary conditions
                    **_tm_reljump('nb_mark_7_noprev_periodic', -size - 1, 'nb_mark_7_noprev_periodic_mark'),
                    **_tm_mark('nb_mark_7_noprev_periodic_mark', 'nb_mark_7_noprev_periodic_ret_0'),
                    **_tm_reljump('nb_mark_7_noprev_periodic_ret', size, 'neighbor_count_start'),
                **_tm_mark('nb_mark_7_noprev', 'neighbor_count_start'),
            # else:
                # neighbor 3
                ('nb_mark_3', 'R'): ('R', 'R', 'nb_mark_3_periodic_0'),
                # periodic boundary conditions
                    **_tm_reljump('nb_mark_3_periodic', size - 1, 'nb_mark_3_wrap'),
                    **_tm_mark('nb_mark_3_wrap', 'nb_mark_3_wrap_ret_0'),
                    **_tm_reljump('nb_mark_3_wrap_ret', -size, 'nb_mark_c'),
                **_tm_mark('nb_mark_3', 'nb_mark_c'),

                # center cell
                **_tm_center_mark('nb_mark_c', 'nb_mark_check_first_row_0'),

                # check if (first row):
                    **_tm_any_transition('nb_mark_check_first_row_0', 'nb_mark_check_first_row_1', 'L'),
                    **_tm_bwd_symjmp('nb_mark_check_first_row_1', ['R', '^'], 'nb_mark_check_first_row_2'),
                    ('nb_mark_check_first_row_2', '^'): ('^', 'R', 'nb_mark_check_first_row_true_ret'), # is in the first row
                    ('nb_mark_check_first_row_2', 'R'): ('R', 'R', 'nb_mark_check_first_row_false_ret'), # is not in the first row
                    **_tm_fwd_symjmp('nb_mark_check_first_row_true_ret', ['CO', 'CX', 'COO', 'COX', 'CXO', 'CXX'], 'nb_mark_4_fr_pre'),
                    **_tm_fwd_symjmp('nb_mark_check_first_row_false_ret', ['CO', 'CX', 'COO', 'COX', 'CXO', 'CXX'], 'nb_mark_check_last_row_0'),
                    # we need to advance by one to get to the correct tape position
                    **_tm_any_transition('nb_mark_4_fr_pre', 'nb_mark_4_noprev'),

                # check if (last row):
                    **_tm_fwd_symjmp('nb_mark_check_last_row_0', ['R', 'E'], 'nb_mark_check_last_row_1'),
                    ('nb_mark_check_last_row_1', 'E'): ('E', 'R', 'nb_mark_check_last_row_true_ret'), # is in the last row
                    ('nb_mark_check_last_row_1', 'R'): ('R', 'R', 'nb_mark_check_last_row_false_ret'), # is not in the last row
                    **_tm_bwd_symjmp('nb_mark_check_last_row_true_ret', ['CO', 'CX', 'COO', 'COX', 'CXO', 'CXX'], 'nb_mark_4_lr_pre'),
                    **_tm_bwd_symjmp('nb_mark_check_last_row_false_ret', ['CO', 'CX', 'COO', 'COX', 'CXO', 'CXX'], 'nb_mark_4_nlr_pre'),
                    # we need to advance by one to get to the correct tape position
                    **_tm_any_transition('nb_mark_4_lr_pre', 'nb_mark_4_no_next'),
                    **_tm_any_transition('nb_mark_4_nlr_pre', 'nb_mark_4'),

                    # neighbor 4 (behaves the same as the bottom right edge-case, with the exception, that we actually mark the cell)
                    # I think this works as expected
                    ('nb_mark_4_no_next', 'E'): ('E', 'R', 'nb_mark_4_no_next_periodic_jmp_0'),
                    # long-wrap periodic boundary conditions
                        **_tm_reljump('nb_mark_4_no_next_periodic_jmp', -size - 1, 'nb_mark_4_no_next_periodic_mark'), # jump to the beginning of the last row
                        **_tm_mark('nb_mark_4_no_next_periodic_mark', 'nb_mark_4_no_next_periodic_next_row_jmp_long'), # mark neighbor 4
                        # jump to the beginning of the grid
                        **_tm_bwd_symjmp('nb_mark_4_no_next_periodic_next_row_jmp_long', ['^'], 'nb_mark_4_no_next_periodic_next_row_jmp_long_start_0'),
                        **_tm_reljump('nb_mark_4_no_next_periodic_next_row_jmp_long_start', size - 1, 'nb_mark_4_no_next_periodic_next_row_mark_5'), # jump to the right point in the first row
                        # mark the neighbors 5, 6 and 7
                        **_tm_mark('nb_mark_4_no_next_periodic_next_row_mark_5', 'nb_mark_4_no_next_periodic_next_row_mark_6'),
                        **_tm_mark('nb_mark_4_no_next_periodic_next_row_mark_6', 'nb_mark_4_no_next_periodic_next_row_mark_7'),
                        **_tm_mark('nb_mark_4_no_next_periodic_next_row_mark_7', 'nb_mark_4_no_next_periodic_next_row_mark_jmp_back'),
                        ('nb_mark_4_no_next_periodic_next_row_mark_7', 'R'): ('R', 'R', 'nb_mark_4_no_next_periodic_next_row_mark_7_periodic_0'),
                        # periodic boundary conditions
                            **_tm_reljump('nb_mark_4_no_next_periodic_next_row_mark_7_periodic', -size - 1, 'nb_mark_4_no_next_periodic_next_row_mark_7_periodic_mark'),
                            # no need to jump back here, as we do it with the fwd symjmp anyways.
                            **_tm_mark('nb_mark_4_no_next_periodic_next_row_mark_7_periodic_mark', 'nb_mark_4_no_next_periodic_next_row_mark_jmp_back'),
                        # jump back to the end of the grid
                        # and continue marking the next neighbors
                        **_tm_fwd_symjmp('nb_mark_4_no_next_periodic_next_row_mark_jmp_back', ['E'], 'nb_mark_jmp_prev_early_1'), # a slight hack, to make sure we jump one less element back
                    # long-jump wrapping for neighbors 5, 6 and 7 is also required in the case, that the 4th neighbors is not the end-element
                    # but in that case, we have to use a huge relative jump (because we need to end up at the relative offset)
                    # TODO: but we can re-use this jump for the other case!
                    **_tm_mark('nb_mark_4_no_next', 'nb_mark_4_no_next_next_row_start_jmp_0'),
                        **_tm_reljump('nb_mark_4_no_next_next_row_start_jmp', -(size*size) + 1 - 3, 'nb_mark_4_no_next_next_row_mark_5'),
                        # mark the neighbors 5, 6 and 7 (with boundary conditions)
                        **_tm_mark('nb_mark_4_no_next_next_row_mark_5', 'nb_mark_4_no_next_next_row_mark_6'),
                        ('nb_mark_4_no_next_next_row_mark_5', '^'): ('^', 'R', 'nb_mark_4_no_next_next_row_mark_5_periodic_0'),
                            **_tm_reljump('nb_mark_4_no_next_next_row_mark_5_periodic', size - 1, 'nb_mark_4_no_next_next_row_mark_5_periodic_mark'),
                            **_tm_mark('nb_mark_4_no_next_next_row_mark_5_periodic_mark', 'nb_mark_4_no_next_next_row_mark_5_periodic_ret_0'),
                            **_tm_reljump('nb_mark_4_no_next_next_row_mark_5_periodic_ret', -size, 'nb_mark_4_no_next_next_row_mark_6'),                        
                        **_tm_mark('nb_mark_4_no_next_next_row_mark_6', 'nb_mark_4_no_next_next_row_mark_7'),
                        **_tm_mark('nb_mark_4_no_next_next_row_mark_7', 'nb_mark_4_no_next_next_row_mark_jmp_back_0'),
                        ('nb_mark_4_no_next_next_row_mark_7', 'R'): ('R', 'R', 'nb_mark_4_no_next_next_row_mark_7_periodic_0'),
                            **_tm_reljump('nb_mark_4_no_next_next_row_mark_7_periodic', -size - 1, 'nb_mark_4_no_next_next_row_mark_7_periodic_mark'),
                            **_tm_mark('nb_mark_4_no_next_next_row_mark_7_periodic_mark', 'nb_mark_4_no_next_next_row_mark_7_periodic_mark_ret_0'),
                            **_tm_reljump('nb_mark_4_no_next_next_row_mark_7_periodic_mark_ret', size, 'nb_mark_4_no_next_next_row_mark_jmp_back_0'),
                        **_tm_reljump('nb_mark_4_no_next_next_row_mark_jmp_back', (size*size) - 1, 'nb_mark_jmp_prev_early_0'),
                    **_tm_reljump('nb_mark_jmp_prev_early', -size - 1 - 3, 'nb_mark_0', stop_symbols=['^']),


                # check if we're at the bottom-right edge of the grid
                # if (bottom-right edge):
                # we jump to the spot for the neighbor 0 case, skipping the next row
                ('nb_mark_4', 'E'): ('E', 'R', 'nb_mark_4_no_next_periodic_0'), # we can re-use the other case here, I think.

                # neighbor 4
                ('nb_mark_4', 'R'): ('R', 'R', 'nb_mark_4_periodic_0'),
                # periodic boundary conditions
                    **_tm_reljump('nb_mark_4_periodic', -size - 1, 'nb_mark_4_periodic_wrap'),
                    **_tm_mark('nb_mark_4_periodic_wrap', 'nb_mark_4_periodic_wrap_ret_0'),
                    **_tm_reljump('nb_mark_4_periodic_wrap_ret', size, 'nb_mark_jmp_next_row_0'),
                **_tm_mark('nb_mark_4', 'nb_mark_jmp_next_row_0'),
                **_tm_reljump('nb_mark_jmp_next_row', size - 2, 'nb_mark_5', stop_symbols=['E']),


            # if (left edge):
            # apply periodic boundary conditions to neighbor 5
            ('nb_mark_5', 'R'): ('R', 'R', 'nb_mark_5_periodic_0'),
                **_tm_reljump('nb_mark_5_periodic', size - 1, 'nb_mark_5_wrap'),
                **_tm_mark('nb_mark_5_wrap', 'nb_mark_5_wrap_ret_0'),
                **_tm_reljump('nb_mark_5_wrap_ret', -size, 'nb_mark_6'),
            **_tm_mark('nb_mark_5', 'nb_mark_6'),

            # neighbor 6
            **_tm_mark('nb_mark_6', 'nb_mark_7'),

            # if (right edge):
            # skip neighbor 7
            ('nb_mark_7', 'E'): ('E', 'R', 'nb_mark_7_periodic_0'),
            ('nb_mark_7', 'R'): ('R', 'R', 'nb_mark_7_periodic_0'),
                **_tm_reljump('nb_mark_7_periodic', -size - 1, 'nb_mark_7_periodic_mark'),
                **_tm_mark('nb_mark_7_periodic_mark', 'nb_mark_7_periodic_ret_0'),
                **_tm_reljump('nb_mark_7_periodic_ret', size, 'nb_mark_jmp_prev_row_0'),
            **_tm_mark('nb_mark_7', 'nb_mark_jmp_prev_row_0'),
            **_tm_reljump('nb_mark_jmp_prev_row', - 2 * size - 5, 'nb_mark_0', stop_symbols=['^']),

            # if (left edge):
            # skip neighbor 0
            ('nb_mark_0', 'R'): ('R', 'R', 'nb_mark_0_periodic_0'),
            ('nb_mark_0', '^'): ('^', 'R', 'nb_mark_0_periodic_0'),
            # periodic boundary conditions
                **_tm_reljump('nb_mark_0_periodic', size - 1, 'nb_mark_0_periodic_wrap'),
                **_tm_mark('nb_mark_0_periodic_wrap', 'nb_mark_0_periodic_wrap_ret_0'),
                **_tm_reljump('nb_mark_0_periodic_wrap_ret', -size, 'nb_mark_1'),
            **_tm_mark('nb_mark_0', 'nb_mark_1'),

            # neighbor 1
            **_tm_mark('nb_mark_1', 'nb_mark_2'),

            # if (right edge):
            # skip neighbor 2
            ('nb_mark_2', 'R'): ('R', 'R', 'nb_mark_2_periodic_0'),
            # periodic boundary conditions
                **_tm_reljump('nb_mark_2_periodic', -size - 1, 'nb_mark_2_wrap'),
                **_tm_mark('nb_mark_2_wrap', 'neighbor_count_start'),
            **_tm_mark('nb_mark_2', 'neighbor_count_start'),
        
        # fn neighbor_count
            **_tm_any_transition('neighbor_count_start', 'neighbor_count'),
            # function to count the # of alive neighbors
            # since this can be called from mostly anywhere, the easiest way to make this consistent
            # is to simple jump to the end of the grid and then start counting backwards to the start
            **_tm_fwd_symjmp('neighbor_count', ['E'], 'neighbor_count_next'), # jumps to the end of the grid
            **_tm_bwd_symjmp('neighbor_count_next', ['X', 'XO', 'XX', '^'], 'neighbor_count_op'),
            ('neighbor_count_op', 'X'): (1, 'R', 'inc_jmp'), # reset the element and increment the counter
            ('neighbor_count_op', 'XX'): ('11', 'R', 'inc_jmp'),
            ('neighbor_count_op', 'XO'): ('10', 'R', 'inc_jmp'),
            ('neighbor_count_op', '^'): ('^', 'R', 'nb_unmark'), # start of the tape, means we're done counting, i.e. we can reset the elements

        # fn inc_jmp
            # function to increment the counter
            **_tm_fwd_symjmp('inc_jmp', '$', 'inc_start'), # jump to the end of the tape
            ('inc_start', '$'): ('$', 'L', 'inc_0'), # go back one symbol, to get to the LSB of the counter
            **_tm_increment('inc', 3), # increment the counter
            **_tm_any_transition('inc_end', 'neighbor_count_next'), # continue with the neighbor counting iterator


        # fn nb_unmark
            # Function to reset the marked neighbors
            # wasteful implementation, (since we jump to the beginning of the tape first)
            # but that means less steps to implement
            **_tm_bwd_symjmp('nb_unmark', ['^'], 'nb_unmark_next'), # jump to the start of the tape
            **_tm_fwd_symjmp('nb_unmark_next', ['O', 'OO', 'OX', 'E'], 'nb_unmark_start'), # jump to the next marked neighbor
            ('nb_unmark_start', 'CO'): ('CO', 'R', 'nb_unmark_start'), # if we land on the center cell, skip it
            ('nb_unmark_start', 'CX'): ('CX', 'R', 'nb_unmark_start'),
            ('nb_unmark_start', 'O'): (0, 'R', 'nb_unmark_end'), # unmark the neighbor
            ('nb_unmark_start', 'OO'): ('00', 'R', 'nb_unmark_end'),
            ('nb_unmark_start', 'OX'): ('01', 'R', 'nb_unmark_end'),
            ('nb_unmark_start', 1): (1, 'R', 'nb_unmark_start'), # skip other cells
            ('nb_unmark_start', 0): (0, 'R', 'nb_unmark_start'),
            ('nb_unmark_start', 'R'): ('R', 'R', 'nb_unmark_start'),
            ('nb_unmark_start', 'E'): ('E', 'R', 'cell_update'), # end of the grid, we're done
        
        # fn cell_update
            # Function to update the cell (but leave it marked for now - so we can return to it, to continue updating the grid)
            # as long as the unmarking function is called before this, it's somewhat efficient
            # since that ends at the 'E' symbol, that marks the end of the grid
            # theoretically, we can look for it, but we're pretty much guaranteed to be just beyond the end of the grid here
            # i.e. the next spot after the 'E' symbol
            # **_tm_fwd_symjmp('cell_update', ['E'], 'cell_update_next'), # ensure we're at the end of the grid
            **_tm_bwd_symjmp('cell_update', ['CX', 'CO'], 'cell_update_start'), # jump back to the center cell
            ('cell_update_start', 'CX'): ('CX', 'R', 'cell_update_set'),
            ('cell_update_start', 'CO'): ('CO', 'R', 'cell_update_unset'),

        # fn cell_update_set
            # Function to compute the update (if it's one)
            # resets the counter as well
            **_tm_fwd_symjmp('cell_update_set', ['$'], 'cell_update_set_jmp_0'), # jump to the end of the tape
            **_tm_reljump('cell_update_set_jmp', -3, 'cell_update_set_start'), # jump to the beginning of the counter
            # if the counter starts with 1, we reset it as we go and then jump to the unset function
            ('cell_update_set_start', 1): (0, 'R', 'cell_update_set_unset_0'),
            ('cell_update_set_unset_0', 0): (0, 'R', 'cell_update_set_unset_1'),
            ('cell_update_set_unset_0', 1): (0, 'R', 'cell_update_set_unset_1'),
            ('cell_update_set_unset_1', 0): (0, 'R', 'cell_update_set_unset_jmp'),
            ('cell_update_set_unset_1', 1): (0, 'R', 'cell_update_set_unset_jmp'),
            # if the counter starts with 0, we need to check the other bits
            ('cell_update_set_start', 0): (0, 'R', 'cell_update_set_step_0'),
            ('cell_update_set_step_0', 1): (0, 'R', 'cell_update_set_step_1_set'),
            ('cell_update_set_step_0', 0): (0, 'R', 'cell_update_set_unset_1'), # if the counter is < 2 -> unset the cell and reset the counter
            ('cell_update_set_step_1_set', 1): (0, 'R', 'cell_update_set_set_jmp'), # 11 -> 3 neighbors -> set the cell
            ('cell_update_set_step_1_set', 0): (0, 'R', 'cell_update_set_set_jmp'), # 10 -> 2 neighbors -> keep the cell set
            **_tm_bwd_symjmp('cell_update_set_unset_jmp', ['CX'], 'cell_update_set_unset'), # jump back to the center cell
            ('cell_update_set_unset', 'CX'): ('10', 'R', 'nb_mark_start'), # unset the cell and continue with the next
            **_tm_bwd_symjmp('cell_update_set_set_jmp', ['CX'], 'cell_update_set_set'), # jump back to the center cell
            ('cell_update_set_set', 'CX'): ('11', 'R', 'nb_mark_start'), # set the cell and continue with the next

        # fn cell_update_unset
            # Function to compute the update (if it's zero)
            **_tm_fwd_symjmp('cell_update_unset', ['$'], 'cell_update_unset_jmp_0'), # jump to the end of the tape
            **_tm_reljump('cell_update_unset_jmp', -3, 'cell_update_unset_start'), # jump to the beginning of the counter
            # if the counter starts with 1, we reset it as we go and then jump to the unset function
            ('cell_update_unset_start', 1): (0, 'R', 'cell_update_unset_unset_0'),
            ('cell_update_unset_unset_0', 0): (0, 'R', 'cell_update_unset_unset_1'),
            ('cell_update_unset_unset_0', 1): (0, 'R', 'cell_update_unset_unset_1'),
            ('cell_update_unset_unset_1', 0): (0, 'R', 'cell_update_unset_unset_jmp'),
            ('cell_update_unset_unset_1', 1): (0, 'R', 'cell_update_unset_unset_jmp'),

            # if the counter starts with 0, we need to check the other bits
            ('cell_update_unset_start', 0): (0, 'R', 'cell_update_unset_step_0'),
            ('cell_update_unset_step_0', 1): (0, 'R', 'cell_update_unset_step_1'),
            ('cell_update_unset_step_0', 0): (0, 'R', 'cell_update_unset_unset_1'), # if the counter is < 2 -> keep the cell unset
            ('cell_update_unset_step_1', 1): (0, 'R', 'cell_update_unset_set_jmp'), # 11 -> 3 neighbors -> set the cell
            ('cell_update_unset_step_1', 0): (0, 'R', 'cell_update_unset_unset_jmp'), # 10 -> 2 neighbors -> keep the cell unset
            **_tm_bwd_symjmp('cell_update_unset_unset_jmp', ['CO'], 'cell_update_unset_unset'), # jump back to the center cell
            ('cell_update_unset_unset', 'CO'): ('00', 'R', 'nb_mark_start'), # unset the cell and continue with the next
            **_tm_bwd_symjmp('cell_update_unset_set_jmp', ['CO'], 'cell_update_unset_set'), # jump back to the center cell
            ('cell_update_unset_set', 'CO'): ('01', 'R', 'nb_mark_start'), # set the cell and continue with the next
        
        # fn cell_collapse
            # Function to collapse the dual-state representations
            # jumps back to the beginning of the tape
            ('cell_collapse', '00'): (0, 'L', 'cell_collapse'),
            ('cell_collapse', '01'): (1, 'L', 'cell_collapse'),
            ('cell_collapse', '10'): (0, 'L', 'cell_collapse'),
            ('cell_collapse', '11'): (1, 'L', 'cell_collapse'),
            ('cell_collapse', 'R'): ('R', 'L', 'cell_collapse'),
            ('cell_collapse', '^'): ('^', 'R', 'done_0'),
            **_tm_any_transition('done_0', 'done', dir='L'), # reset back to the beginning of the tape (for restarting the machine!)
    }

    return states

In [ ]:

visualize_intermediates = False

size = 10
state = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
    0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
    0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]

tape = create_tm_gol_tape(size, state)

blank_symbol = ' '
initial_state = 'start_0'
final_states = {'done'}
if visualize_intermediates:
    final_states.add('neighbor_count_start')

tm = TuringMachine(tape, blank_symbol, initial_state, final_states, _make_tm_gol(size))

fig = plt.figure()
camera = Camera(fig)

visualize_tm_gol_state(tm.tape, size)
camera.snap()

for _ in range(50):
    tm.current_state = 'start_0'
    while tm.current_state != 'done':
        tm.run(debug=False)
        visualize_tm_gol_state_intermediate(tm.tape, size)
        camera.snap()

    # # print(tm.tape, tm.current_state)
    # tm.run(debug=False)
    visualize_tm_gol_state(tm.tape, size)
    camera.snap()
    # # print(tm.tape, tm.current_state)

animation = camera.animate()
HTML(animation.to_html5_video())